<h1><center>ARES Evaluation Strategies</h1></center>

<h2>IDP + UES</h2>
<p>Uses targeted prompts to enable pre-trained models to assess content relevance and accuracy in a zero-shot manner.</p>

In [ ]:
from ares import ARES

ues_idp_config = {
    # Dataset for in-domain prompts
    "in_domain_prompts_dataset": "../datasets/multirc_few_shot_prompt_for_synthetic_query_generation_v1.tsv",
    
    # Dataset for unlabeled evaluation
    "unlabeled_evaluation_set": "../datasets_v2/nq/ratio_0.6_reformatted_full_articles_False_validation_with_negatives.tsv", 
    
    # Default model choice
    "model_choice" : "gpt-3.5-turbo-1106"
}

# Optional: Provide an alternative model of your choice below.
# Here are some models you can choose from:
# - mistralai/Mistral-7B-Instruct-v0.2
# - mistralai/Mixtral-8x7B-Instruct-v0.1
# - gpt-4-turbo-preview
# - microsoft/deberta-v3-large
# - openlm-research/open_llama_7b_v2
# - mosaicml/mpt-7b-instruct

In [2]:
ares_module = ARES(ues_idp=ues_idp_config)
results = ares_module.ues_idp()
print(results)

{'Context Relevance Scores': 0.25, 'Answer Faithfulness Scores': 0.005, 'Answer Relevance Scores': 0.004}


<h2>Training Classifier + UES</h2>

In [ ]:
from ares import ARES

ues_idp_config = {
    # Dataset for in-domain prompts
    "in_domain_prompts_dataset": "./datasets/multirc_few_shot_prompt_for_synthetic_query_generation_v1.tsv",
    
    # Dataset for unlabeled evaluation
    "unlabeled_evaluation_set": "./datasets_v2/nq/ratio_0.5_reformatted_full_articles_False_validation_with_negatives.tsv", 
    
    # Default model choice
    "model_choice": "gpt-3.5-turbo-1106"
}

classifier_config = {
    "classification_dataset": "output/synthetic_queries_1.tsv", 
    "test_set_selection": "./datasets_v2/nq/ratio_0.5_reformatted_full_articles_False_validation_with_negatives.tsv", #UES File
    "label_column": "Context_Relevance_Label", 
    "num_epochs": 10, 
    "patience_value": 3, 
    "learning_rate": 5e-6
}

In [3]:
ares_modile = ARES(classifier_model=classifier_config)
results = ares_module.train_classifier()
print(results)

Accuracy for Test Set: 0.8137082601054482


<h2>Training Classifier + PPI + UES</h2>

<h3>UES</h3>

In [ ]:
from ares import ARES

ues_idp_config = {
    # Dataset for in-domain prompts
    "in_domain_prompts_dataset": "../datasets/multirc_few_shot_prompt_for_synthetic_query_generation_v1.tsv",
    
    # Dataset for unlabeled evaluation
    "unlabeled_evaluation_set": "../datasets_v2/nq/ratio_0.5_reformatted_full_articles_False_validation_with_negatives.tsv", 
    
    # Default model choice
    "model_choice" : "gpt-3.5-turbo-1106"
}

In [ ]:
ares = ARES(ues_idp=ues_idp_config)
results = ares.ues_idp()
print(results)

<h3>Training Classifier</h3>

<p>Generates checkpoint which is used in PPI below</p>

In [ ]:
from ares import ARES

classifier_config = {
    "classification_dataset": "output/synthetic_queries_1.tsv", 
    "test_set_selection": "./datasets_v2/nq/ratio_0.5_reformatted_full_articles_False_validation_with_negatives.tsv", #UES File
    "label_column": "Context_Relevance_Label", 
    "model_choice": "microsoft/deberta-v3-large",
    "num_epochs": 10, 
    "patience_value": 3, 
    "learning_rate": 5e-6
}

In [ ]:
ares = ARES(classifier_model=classifier_config)
results = ares.train_classifier()
print(results)

<h3>PPI</h3>

In [1]:
from ares import ARES
import warnings

warnings.filterwarnings("ignore")

ppi_config = { 
    "evaluation_datasets": ['../data/datasets_v2/nq/nq_ratio_0.6_.tsv'], 
    "few_shot_examples_filepath": "../data/datasets/multirc_few_shot_prompt_for_synthetic_query_generation_v1.tsv",
    "checkpoints": ["../data/checkpoints/microsoft-deberta-v3-large/output-synthetic_queries_1.tsv/5e-06_1_True_Context_Relevance_Label_ratio_0.6_reformatted_full_articles_False_validation_with_negatives_428380.pt", "../data/checkpoints/microsoft-deberta-v3-large/output-synthetic_queries_1.tsv/5e-06_1_True_Answer_Relevance_Label_ratio_0.6_reformatted_full_articles_False_validation_with_negatives_428380.pt"],
    "labels": ["Context_Relevance_Label", "Answer_Relevance_Label"], 
    "GPT_scoring": False, 
    "gold_label_path": "../data/datasets_v2/nq/nq_ratio_0.6_.tsv", 
    "swap_human_labels_for_gpt4_labels": False
}


In [4]:
ares_module = ARES(ppi=ppi_config)
results = ares_module.evaluate_RAG()
print(results)

--------------------------------------------------------
Evaluation Sets: ['../data/datasets_v2/nq/nq_ratio_0.6_']
Checkpoints: ['../data/checkpoints/microsoft-deberta-v3-large/output-synthetic_queries_1.tsv/5e-06_1_True_Context_Relevance_Label_ratio_0.6_reformatted_full_articles_False_validation_with_negatives_428380.pt', '../data/checkpoints/microsoft-deberta-v3-large/output-synthetic_queries_1.tsv/5e-06_1_True_Answer_Relevance_Label_ratio_0.6_reformatted_full_articles_False_validation_with_negatives_428380.pt']
Labels: ['Context_Relevance_Label', 'Answer_Relevance_Label']
GPT Scoring: False
--------------------------------------------------------


FileNotFoundError: [Errno 2] No such file or directory: '../data/datasets_v2/nq/nq_ratio_0.6_'

<h2>ARES Comparison to RAGAS and Zeroshot Llama</h2>

<h3>ARES Configuration</h3>

<p>Synthetic Generator</p>

In [ ]:
from ares import ARES
    
synth_config = { 
    "document_filepath": "datasets_v2/nq/ratio_0.5_reformatted_full_articles_False_validation_with_negatives.tsv",
    "few_shot_prompt_filename": "datasets/multirc_few_shot_prompt_for_synthetic_query_generation_v1.tsv",
    "synthetic_queries_filename": "output/synthetic_queries_1.tsv",
    "model_choice": "google/flan-t5-xxl", # Default Model Choice, provide alternative model as you wish. 
    "documents_sampled": 6381
}

ares = ARES(synthetic_query_generator=synth_config)
results = ares.generate_synthetic_data()
print(results)

<p>Training Classifier</p>

In [ ]:
from ares import ARES

classifier_config = {
    "classification_dataset": "output/synthetic_queries_1.tsv", 
    "test_set_selection": "../datasets_v2/nq/ratio_0.6_reformatted_full_articles_False_validation_with_negatives.tsv", 
    "label_column": "Answer_Relevance_Label", 
    "model_choice": "microsoft/deberta-v3-large", # Default Model Choice, provide alternative model as you wish. 
    "num_epochs": 10, 
    "patience_value": 3, 
    "learning_rate": 5e-6
}

ares = ARES(classifier_model=classifier_config)
results = ares.train_classifier()
print(results)

<p>PPI</p>

In [8]:
from ares import ARES

ppi_config = { 
    "evaluation_datasets": ['../data/datasets_v2/nq/ratio_0.6_reformatted_full_articles_False_validation_with_negatives.tsv'], 
    "few_shot_examples_filepath": "../data/datasets/multirc_few_shot_prompt_for_synthetic_query_generation_v1.tsv",
    "checkpoints": ["../data/checkpoints/microsoft-deberta-v3-large/output-synthetic_queries_1.tsv/5e-06_1_True_Context_Relevance_Label_ratio_0.6_reformatted_full_articles_False_validation_with_negatives_428380.pt", "/future/u/manihani/ARES/data/checkpoints/microsoft-deberta-v3-large/output-synthetic_queries_1.tsv/5e-06_1_True_Answer_Relevance_Label_ratio_0.6_reformatted_full_articles_False_validation_with_negatives_428380.pt"],
    "labels": ["Context_Relevance_Label", "Answer_Relevance_Label"], 
    "model_choice": "microsoft/deberta-v3-large", # Default Model Choice, provide alternative model as you wish. 
    "GPT_scoring": False, 
    "gold_label_path": "../data/datasets_v2/nq/ratio_0.6_reformatted_full_articles_False_validation_with_negatives.tsv", 
    "swap_human_labels_for_gpt4_labels": False
}

ares_module = ARES(ppi=ppi_config)
results = ares_module.evaluate_RAG()
print(results)

 
--------------------------------------------------
Context_Relevance_Label Scoring
ARES Ranking
[0]
Avg. PPIs: [0.6157347655943952]
PPI Confidence Intervals: [[0.5422749567760038, 0.6891945744127868]]
Evaluation Set Lengths: [4421]
Evaluation Set Ratio: [0.6]
Test Accuracy Scores: [0.7733544446957702]
Y-Labeled Example Count: 153
--------------------------------------------------


--------------------------------------------------
Answer_Relevance_Label Scoring
ARES Ranking
[0]
Avg. PPIs: [0.6066581096943275]
PPI Confidence Intervals: [[0.5541757588683177, 0.6591404605203375]]
Evaluation Set Lengths: [4421]
Evaluation Set Ratio: [0.6]
Test Accuracy Scores: [0.8787604614340647]
Y-Labeled Example Count: 169
--------------------------------------------------


<h3>RAGAS Configuration</h3>

<p>Data Cleaning | Context Relevance Label Filter</p>

In [10]:
from datasets import load_dataset, Dataset
import pandas as pd
import os

def load_and_prepare_dataset(file_path):
    # Load the dataset from the TSV file
    dataset_df = pd.read_csv(file_path, delimiter='\t')
    
    # Convert 'Context_Relevance_Label' to string if it is not already
    dataset_df['Context_Relevance_Label'] = dataset_df['Context_Relevance_Label'].astype(str)
    
    # Use 'Context_Relevance_Label' as 'ground_truth'
    prepared_data = {
        'question': dataset_df['Query'].tolist(),
        'contexts': [[doc] for doc in dataset_df['Document'].tolist()],  # Contexts are expected to be list of lists
        'answer': dataset_df['Answer'].tolist(),
        'ground_truth': dataset_df['Context_Relevance_Label'].tolist(),  # Using 'Context_Relevance_Label' as 'ground_truth'
    }
    
    # Convert to HuggingFace's Dataset format
    dataset = Dataset.from_dict(prepared_data)
    return dataset


<p>Context Relevance Accuracy</p>

In [12]:
from ragas import evaluate
from ragas.metrics import faithfulness, context_recall, context_precision
import os

file_path = "./datasets_v2/nq/ratio_0.6_reformatted_full_articles_False_validation_with_negatives.tsv"  # Update this path
prepared_dataset = load_and_prepare_dataset(file_path)

# Specify metrics
metrics = [
    # Add or remove metrics based on your evaluation plan
    context_precision,
    context_recall,
]

# Evaluate
result = evaluate(prepared_dataset, metrics=[context_precision, context_recall])  # Update metrics as needed

# Assuming 'result' can be explored or exported as needed
print(result)

Evaluating:   0%|          | 0/12378 [00:00<?, ?it/s]

{'context_precision': 0.4424, 'context_recall': 0.4830}


<p>Data Cleaning | Answer Faithfulness Label Filter</p>

In [16]:
from datasets import load_dataset, Dataset
import pandas as pd
import os

def load_and_prepare_dataset(file_path):
    # Load the dataset from the TSV file
    dataset_df = pd.read_csv(file_path, delimiter='\t')
    
    # Convert 'Context_Relevance_Label' to string if it is not already
    dataset_df['Answer_Faithfulness_Label'] = dataset_df['Answer_Faithfulness_Label'].astype(str)
    
    # Use 'Context_Relevance_Label' as 'ground_truth'
    prepared_data = {
        'question': dataset_df['Query'].tolist(),
        'contexts': [[doc] for doc in dataset_df['Document'].tolist()], 
        'answer': dataset_df['Answer'].tolist(),
        'ground_truth': dataset_df['Answer_Faithfulness_Label'].tolist(), 
    }
    
    # Convert to HuggingFace's Dataset format
    dataset = Dataset.from_dict(prepared_data)
    return dataset


In [ ]:
from ragas import evaluate
from ragas.metrics import faithfulness, context_recall, context_precision
import os

file_path = "./datasets_v2/nq/ratio_0.6_reformatted_full_articles_False_validation_with_negatives.tsv" 
prepared_dataset = load_and_prepare_dataset(file_path)

# Specify metrics
metrics = [
    faithfulness
]

# Evaluate
result = evaluate(prepared_dataset, metrics=[faithfulness]) 

print(result)

<h3>Zeroshot Llama Configuration</h3>

In [1]:
from ares import ARES


ues_idp_config = {
    # Dataset for in-domain prompts
    "in_domain_prompts_dataset": "../datasets/multirc_few_shot_prompt_for_synthetic_query_generation_v1.tsv",
    
    # Dataset for unlabeled evaluation
    "unlabeled_evaluation_set": "../datasets_v2/nq/ratio_0.6_reformatted_full_articles_False_validation_with_negatives.tsv", 
    
    # Model: Mistral 7B
    "model_choice" : "codellama/CodeLlama-13b-Instruct-hf"
}

/future/u/manihani/miniconda3/envs/llm_judge/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
ares = ARES(ues_idp=ues_idp_config)
results = ares.ues_idp()
print(results)

Didn't extract Yes or No!
Didn't extract Yes or No!
Didn't extract Yes or No!
Didn't extract Yes or No!
Didn't extract Yes or No!
Didn't extract Yes or No!
Didn't extract Yes or No!
Didn't extract Yes or No!
Didn't extract Yes or No!
Didn't extract Yes or No!
Didn't extract Yes or No!
{'Context Relevance Scores': 0.94, 'Answer Faithfulness Scores': 0.013, 'Answer Relevance Scores': 0.012}


<h3>Zeroshot Mistral Configuration</h3>

In [8]:
from ares import ARES
import os

ues_idp_config = {
    # Dataset for in-domain prompts
    "in_domain_prompts_dataset": "../data/datasets/multirc_few_shot_prompt_for_synthetic_query_generation_v1.tsv",
    
    # Dataset for unlabeled evaluation
    "unlabeled_evaluation_set": "../data/datasets_v2/nq/ratio_0.6_reformatted_full_articles_False_validation_with_negatives.tsv", 
    
    # Model: Mistral 7B
    "model_choice" : "mistralai/Mixtral-8x7B-v0.1"
}

In [9]:
ares = ARES(ues_idp=ues_idp_config)
results = ares.ues_idp()
print(results)

InternalServerError: <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.together.xyz | 524: A timeout occurred</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2 font-light text-60 lg:text-4xl text-